In [1]:
# 필요한 라이브러리 설치
!pip install -q transformers chromadb langchain-community google-api-python-client huggingface_hub sentence-transformers

In [7]:
from google.colab import userdata

# Colab Secrets에서 인증 정보 로드
HF_TOKEN = userdata.get('HF_TOKEN') # Hugging Face 액세스 토큰
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Google Custom Search API 키
GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID') # Programmable Search Engine ID

In [3]:
# 1. 데이터 수집 및 벡터 저장소 구축 (NIH DSLD 사용)
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
# NIH 식이보충제 라벨 데이터베이스에서 데이터 로드
loader = WebBaseLoader(["https://dsld.od.nih.gov/dsld/"])
documents = loader.load()

In [5]:
# 문서 분할 (청크 크기 512, 50% 오버랩)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=256
)
texts = text_splitter.split_documents(documents)

# 임베딩 모델 초기화 (gte-base 사용)
embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-base")

# ChromaDB에 벡터 저장
vector_db = Chroma.from_documents(
    texts, embeddings,
    persist_directory="./chroma_db",
    collection_name="supplement_info"
)

<ipython-input-5-319aacbaa380>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-base")


In [8]:
# 2. Google Programmable Search Engine 설정
from googleapiclient.discovery import build

def google_search(query, num=3):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num).execute()
    return [item["snippet"] for item in res["items"]]

# 3. Gemma-3B 모델 초기화
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=HF_TOKEN
)

In [9]:
# 4. 고급 프롬프트 엔지니어링 템플릿
REACT_PROMPT = """<start_of_turn>user
사용자 정보:
{user_info}

현재 단계: {step}
과거 관측: {history}

다음 단계에서 수행해야 할 작업을 다음 형식 중 하나로 출력:
1. 검색 필요: "Action: search[검색어]"
2. DB 조회: "Action: lookup[질문]"
3. 최종 답변: "Final Answer: [답변]"

규칙:
- 의학적 조언은 공인된 출처를 반드시 인용
- 복용량 계산 시 사용자의 신체 조건 고려
- 상호작용 가능성 경고<end_of_turn>
<start_of_turn>assistant
"""

In [10]:
# 5. RAG 에이전트 실행 로직
def run_agent(user_input, max_steps=5):
    history = []
    for step in range(max_steps):
        # ReAct 프롬프트 구성
        prompt = REACT_PROMPT.format(
            user_info=user_input,
            step=step+1,
            history="\n".join(history[-3:])
        )

        # 모델 추론
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=500)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 액션 파싱
        if "Final Answer:" in response:
            print(f"\nStep {step+1}: 최종 답변 생성")
            return response.split("Final Answer:")[-1].strip()

        # 액션 실행 및 관측 기록
        action_type, query = parse_action(response)
        observation = execute_action(action_type, query)

        print(f"\nStep {step+1}:")
        print(f"생각: {response.split('Action:')[0].strip()}")
        print(f"액션: {action_type}({query})")
        print(f"관측: {observation[:200]}...")

        history.append(f"Step {step+1}: {observation[:200]}")

# 액션 처리 함수
def parse_action(response):
    action_line = response.split("Action:")[-1].split("\n")[0].strip()
    if "search[" in action_line:
        return "search", action_line.split("[")[1].split("]")[0]
    elif "lookup[" in action_line:
        return "lookup", action_line.split("[")[1].split("]")[0]
    return "unknown", ""

def execute_action(action_type, query):
    if action_type == "search":
        return "\n".join(google_search(query))
    elif action_type == "lookup":
        docs = vector_db.similarity_search(query, k=3)
        return "\n".join([d.page_content for d in docs])
    return "No action performed"


In [12]:
# 6. 실행 예시
user_input = """
30대 직장인인데 스트레스가 잘 쌓이고 쉽게 피곤해집니다.
잘 지치니 기분도 별로에요. 어떤 영양제 제품을 먹으면 좋을지 추천해주세요.
답변은 한글로 작성해주세요.
"""

result = run_agent(user_input)
print("\n최종 추천:")
print(result)



Step 1: 최종 답변 생성

최종 추천:
[추천 영양제] (추천 이유 및 복용량은 약사 또는 의사와 상담 후 결정해주세요.)"**

**추천 영양제 (참고):**

*   **마그네슘:** 스트레스 해소에 도움을 줄 수 있으며, 근육 이완 및 신경 안정에 효과적입니다.
    *   **추천 제품:**  마그네슘 옥실 있다는 제품 (하루 100mg - 300mg 복용)
    *   **참고:**  마그네슘은 체내 수분 균형을 조절하고 신경 기능을 개선하여 스트레스 완화에 도움을 줄 수 있습니다.
*   **비타민 B군:** 신경 안정 및 스트레스 호르몬 조절에 중요합니다.
    *   **추천 제품:**  비타민 B12, 비타민 B6, 비타민 B1 (하루 100mg - 200mg 복용)
    *   **참고:** 비타민 B군은 신경 기능을 유지하고 스트레스에 대한 저항력을 높이는 데 도움을 줄 수 있습니다.
*   **오메가-3 지방산:** 항염증 효과가 있어 스트레스 관련 염증을 완화하고 심리적 안정에 도움을 줄 수 있습니다.
    *   **
